<a href="https://colab.research.google.com/github/betegon/16-QAM-Detection-MIMO/blob/master/TAC_MIMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np


## EJEMPLO CON 2 antenas


In [0]:
k = 2 # 2 antennas

In [0]:
# select a random sample without replacement
from random import seed
from random import sample

def randomsimb():
  # seed random number generator
  # prepare a sequence
  sequence = [-3, -1, 1, 3]
  # select a subset without replacement
  subset = sample(sequence, 4)
  return np.expand_dims(subset, -1)

In [20]:
s = randomsimb()
print("forma de S",s.shape)

# s_matrix = np.array([[1, 1],[1, 1]])
# print(s_matrix)
# s = (np.array([s_matrix.flatten('F')])).transpose()
# randomsimb()
print("s\n",s)
t = s**2
print(t)

forma de S (4, 1)
s
 [[ 1]
 [-1]
 [-3]
 [ 3]]
[[1]
 [1]
 [9]
 [9]]


In [21]:
# CREACIÓN DE H 

k = 2  # number of antennas
# H is CN(0,1)
mu = np.zeros(2*k)
sigma = np.ones((k,k*2))


H = (1/np.sqrt(2))*np.random.normal(loc=mu, scale=sigma, size=(k,k*2)).view(np.complex128) 
print(H)
a = np.column_stack((np.real(H), -np.imag(H)))
print("Fila1: \n",a,"\n")
b = np.column_stack((np.imag(H), np.real(H)))
print("Fila2\n",b,"\n")
H_expanded = np.concatenate((a,b), axis=0)
print("Matriz H expandida\n",H_expanded)

[[ 1.0299355 +0.55688216j  1.0725703 +0.53126924j]
 [-0.3908438 +0.41231974j -0.25225544-0.65073368j]]
Fila1: 
 [[ 1.0299355   1.0725703  -0.55688216 -0.53126924]
 [-0.3908438  -0.25225544 -0.41231974  0.65073368]] 

Fila2
 [[ 0.55688216  0.53126924  1.0299355   1.0725703 ]
 [ 0.41231974 -0.65073368 -0.3908438  -0.25225544]] 

Matriz H expandida
 [[ 1.0299355   1.0725703  -0.55688216 -0.53126924]
 [-0.3908438  -0.25225544 -0.41231974  0.65073368]
 [ 0.55688216  0.53126924  1.0299355   1.0725703 ]
 [ 0.41231974 -0.65073368 -0.3908438  -0.25225544]]


In [0]:
def awgn(signal, snr, seed=False):
    """ Generate Aditive White Gaussian Noise (AWGN) from an input signal.

    Args:
        signal  (np.array): Input signal. e.g. signal = np.array([[1, 2.3, -1]])
        snr        (float): Signal-to-noise ratio in dB.
        seed        (bool): Set seed to reproduce results (good for debugging).
                            Defaults to False.

    Returns:
        noise (np.array): AWGN generated from an input signal.

    """
    if seed: np.random.seed(1)
    sigpower = sum([ math.pow(abs(signal[i]),2) for i in range(len(signal)) ])
    sigpower = sigpower / len(signal)
    noisepower = sigpower / (math.pow(10,snr/10))
    noise = np.random.normal(0, np.sqrt(noisepower), len(signal))
    return np.array([noise]).T

In [23]:
# y = H*s+w;      
mu = np.zeros(k)
sigma = np.ones((k,k))
snr = 20

noise = awgn(s,snr)
# noise = np.random.normal(loc=mu, scale=sigma, size=(k,k)).view(np.complex128) 
# noise = np.concatenate((np.real(noise), np.imag(noise)), axis=0)



y = H_expanded.dot(s) + noise
print("\nH\n",H_expanded)
print("\ns\n",s)
# print("\nnoise\n",noise)
print("\nnoise \n", noise)
print("\ny\n",y)


H
 [[ 1.0299355   1.0725703  -0.55688216 -0.53126924]
 [-0.3908438  -0.25225544 -0.41231974  0.65073368]
 [ 0.55688216  0.53126924  1.0299355   1.0725703 ]
 [ 0.41231974 -0.65073368 -0.3908438  -0.25225544]]

s
 [[ 1]
 [-1]
 [-3]
 [ 3]]

noise 
 [[-0.14264537]
 [ 0.42334553]
 [-0.51553885]
 [ 0.07613717]]

y
 [[-0.10844142]
 [ 3.47391743]
 [-0.36202153]
 [ 1.55495566]]


In [24]:
## CALCULO DE W (variable a minimizar)
w = np.concatenate((s,t,[[1]]))
w_transpose = w.transpose()
print("w\n",w)
print("w_transpose\n",w_transpose)

W = w.dot(w_transpose)
print("\nBig W\n",W)
print("\nBig W\n",W.shape)

w
 [[ 1]
 [-1]
 [-3]
 [ 3]
 [ 1]
 [ 1]
 [ 9]
 [ 9]
 [ 1]]
w_transpose
 [[ 1 -1 -3  3  1  1  9  9  1]]

Big W
 [[  1  -1  -3   3   1   1   9   9   1]
 [ -1   1   3  -3  -1  -1  -9  -9  -1]
 [ -3   3   9  -9  -3  -3 -27 -27  -3]
 [  3  -3  -9   9   3   3  27  27   3]
 [  1  -1  -3   3   1   1   9   9   1]
 [  1  -1  -3   3   1   1   9   9   1]
 [  9  -9 -27  27   9   9  81  81   9]
 [  9  -9 -27  27   9   9  81  81   9]
 [  1  -1  -3   3   1   1   9   9   1]]

Big W
 (9, 9)


In [25]:
# Cálculo de la matriz a multiplicar por H

A11 = H_expanded.T.dot(H_expanded) # Check if it is really doing the transpose of H_expanded
A12 = np.zeros((2*k,2*k))
A13 = -H_expanded.T.dot(y) # Check if it is really doing the transpose of H_expanded
A1X = np.column_stack((np.column_stack((A11, A12)), A13))
print("\nFILA 1")
print("\n",A1X)

print("\nFILA 2")
A2X = np.zeros((2*k,A1X.shape[1]))
print("\n",A2X)

print("\nFILA 3")
A31 = (-y.T).dot(H_expanded)
A32= np.zeros((1,2*k))
A33 = (y.T).dot(y)
A3X = np.column_stack((np.column_stack((A31, A32)), A33))
A3X = np.column_stack((A31, A32, A33))
print("\n",A3X)

A = np.concatenate((A1X,A2X,A3X))
print("\n Matriz A")
print("\n",A)
print("\n",A.shape)

# A = [H'*H zeros(2*N,2*N) -H'*y; zeros(2*N,4*N+1); -y'*H zeros(1,2*N) y'*y];



FILA 1

 [[ 1.69365131e+00  1.23081472e+00  4.58550766e-17 -3.08222905e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.02991116e+00]
 [ 1.23081472e+00  1.91974118e+00  3.08222905e-01  7.28745190e-17
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.19681853e+00]
 [ 4.58550766e-17  3.08222905e-01  1.69365131e+00  1.23081472e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.35257923e+00]
 [-3.08222905e-01  7.28745190e-17  1.23081472e+00  1.91974118e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.53766709e+00]]

FILA 2

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

FILA 3

 [[ 1.02991116  2.19681853  2.35257923 -1.53766709  0.          0.
   0.          0.         14.62880851]]

 Matriz A

 [[ 1.69365131e+00  1.23081472e+00  4.58550766e-17 -3.08222905e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.000000

In [26]:
import cvxpy as cp
# Remove exponential notation
# np.set_printoptions(suppress=True)

# Problem data.
np.random.seed(1)
k = 2 # number of antennas (TX y RX)
zero_array = np.zeros(2*k)

# Construct the problem.
W = cp.Variable((9,9), PSD=True)

objective = cp.Minimize(cp.trace(cp.matmul(W,A)))
constraints = [W >> 0, 
               cp.diag(W[0:2*k, 0:2*k]) - W[2*k:4*k,4*k] == 0,
               cp.diag(W[2*k:4*k, 2*k:4*k]) - 10*W[2*k:4*k,4*k] + 9*np.ones((2*k)) == 0,
               W[4*k, 4*k] == 1 ]
'''
subject to:
1.  W >> 0 (semidefinida positiva)
2. diag(W(1:2*N,1:2*N)) - W(2*N+1:4*N,4*N+1) == 0
'''
# formulate and solve problem
prob = cp.Problem(objective, constraints)
prob.solve()

# print("Optimal value", prob.solve())
print("Optimal value of W")
print(W.value) # A numpy ndarray.

Optimal value of W
[[  0.99998123  -1.30350731  -2.99976433   2.88534764   1.00005156
    1.69921155   8.99930096   8.32565148   0.9999339 ]
 [ -1.30350731   1.69916933   3.91035306  -3.76120091  -1.30360711
   -2.2148766  -11.73106227 -10.85262642  -1.30346688]
 [ -2.99976433   3.91035306   8.9989547   -8.65567559  -2.99998838
   -5.09712919 -26.99676421 -24.97616901  -2.99967825]
 [  2.88534764  -3.76120091  -8.65567559   8.32550505   2.88556574
    4.90266482  25.9670133   24.02328009   2.88526084]
 [  1.00005156  -1.30360711  -2.99998838   2.88556574   1.00014632
    1.69677141   8.99996737   8.33674616   1.00001156]
 [  1.69921155  -2.2148766   -5.09712919   4.90266482   1.69677141
    7.99113818  15.29398296  16.29279943   1.69912366]
 [  8.99930096 -11.73106227 -26.99676421  25.9670133    8.99996737
   15.29398296  80.99024501  74.92335904   8.99902221]
 [  8.32565148 -10.85262642 -24.97616901  24.02328009   8.33674616
   16.29279943  74.92335904  74.2545873    8.32547328]
 [  0

In [27]:
W11=W.value[0:2*k,0:2*k]
print("Esto es W11:",W11)
print("\n")
W13=W.value[0:2*k,4*k]
print("Estos es W13:",W13)
print("\n")
W31=W.value[4*k,0:2*k]
print("Estos es W31:",W31)
print("\n")
W_1113=np.column_stack((W11,W13))
print("Column stack",W_1113)
one=np.append(W31,1)
one = np.array([one])
print("\n")
print(one)
print("\n")
print(W_1113.shape)
print(one.shape)
W_ED = np.concatenate((W_1113, one))
print("W_ED\n",W_ED)

Esto es W11: [[ 0.99998123 -1.30350731 -2.99976433  2.88534764]
 [-1.30350731  1.69916933  3.91035306 -3.76120091]
 [-2.99976433  3.91035306  8.9989547  -8.65567559]
 [ 2.88534764 -3.76120091 -8.65567559  8.32550505]]


Estos es W13: [ 0.9999339  -1.30346688 -2.99967825  2.88526084]


Estos es W31: [ 0.9999339  -1.30346688 -2.99967825  2.88526084]


Column stack [[ 0.99998123 -1.30350731 -2.99976433  2.88534764  0.9999339 ]
 [-1.30350731  1.69916933  3.91035306 -3.76120091 -1.30346688]
 [-2.99976433  3.91035306  8.9989547  -8.65567559 -2.99967825]
 [ 2.88534764 -3.76120091 -8.65567559  8.32550505  2.88526084]]


[[ 0.9999339  -1.30346688 -2.99967825  2.88526084  1.        ]]


(4, 5)
(1, 5)
W_ED
 [[ 0.99998123 -1.30350731 -2.99976433  2.88534764  0.9999339 ]
 [-1.30350731  1.69916933  3.91035306 -3.76120091 -1.30346688]
 [-2.99976433  3.91035306  8.9989547  -8.65567559 -2.99967825]
 [ 2.88534764 -3.76120091 -8.65567559  8.32550505  2.88526084]
 [ 0.9999339  -1.30346688 -2.99967825  2.8

In [0]:
#quantiz version BT
def quantiz(entry, symbols):
    result = np.empty((len(entry),1))
    for i in range(len(entry)):
        minimum = float("inf")
        for val in symbols:
            if abs(val - entry[i]) < minimum:
                result[i,0] = val
                minimum = abs(val - entry[i])
    return result

In [29]:
#simple quantization 

valores=W.value[0:2*k,4*k]
simple_quantiz = quantiz(valores,s)
print(simple_quantiz)

[[ 1.]
 [-1.]
 [-3.]
 [ 3.]]


In [30]:
#eigenvalue descomposition

u,S,v=np.linalg.svd(W_ED)#este te calcula svd directamente
print (W_ED)
print ("\n")
print(v)
print ("\n")
print(v[0,0:2*k])
print ("\n")
print(v[0,2*k])
eigen=v[0,0:2*k]/v[0,2*k]
eigen_trans=eigen.transpose()
print("\n")
print("ESto es el cociente:",eigen)
print("\n")
simb=[-3,-1, 1 ,3]
eigen_descomposition = quantiz(eigen_trans,simb)


[[ 0.99998123 -1.30350731 -2.99976433  2.88534764  0.9999339 ]
 [-1.30350731  1.69916933  3.91035306 -3.76120091 -1.30346688]
 [-2.99976433  3.91035306  8.9989547  -8.65567559 -2.99967825]
 [ 2.88534764 -3.76120091 -8.65567559  8.32550505  2.88526084]
 [ 0.9999339  -1.30346688 -2.99967825  2.88526084  1.        ]]


[[-2.18092025e-01  2.84293962e-01  6.54249128e-01 -6.29292630e-01
  -2.18086396e-01]
 [-5.73787741e-02  4.89429560e-02  1.85053508e-01 -1.03287340e-01
   9.74371176e-01]
 [-8.18487461e-01  1.13844369e-01 -5.20502445e-01 -2.13772586e-01
   2.22760228e-02]
 [ 2.93275003e-02  9.17658690e-01 -8.16330179e-03  3.96200856e-01
  -8.17935367e-04]
 [ 5.27599960e-01  2.48442184e-01 -5.16453137e-01 -6.25018939e-01
   5.04206882e-02]]


[-0.21809203  0.28429396  0.65424913 -0.62929263]


-0.2180863963336822


ESto es el cociente: [ 1.00002581 -1.30358412 -2.99995387  2.88551987]




In [31]:
from numpy import linalg as la

def nearestPD(A):
    """Find the nearest positive-definite matrix to input
    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].
    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd
    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """

    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    # The above is different from [1]. It appears that MATLAB's `chol` Cholesky
    # decomposition will accept matrixes with exactly 0-eigenvalue, whereas
    # Numpy's will not. So where [1] uses `eps(mineig)` (where `eps` is Matlab
    # for `np.spacing`), we use the above definition. CAVEAT: our `spacing`
    # will be much larger than [1]'s `eps(mineig)`, since `mineig` is usually on
    # the order of 1e-16, and `eps(1e-16)` is on the order of 1e-34, whereas
    # `spacing` will, for Gaussian random matrixes of small dimension, be on
    # othe order of 1e-16. In practice, both ways converge, as the unit test
    # below suggests.
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3

def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False

if __name__ == '__main__':
    import numpy as np
    for i in range(10):
        for j in range(2, 100):
            A = np.random.randn(j, j)
            B = nearestPD(A)
            assert(isPD(B))
    print('unit test passed!')

unit test passed!


In [32]:
#randomization 
#from scipy.linalg import cholesky
#v2=cholesky(W_ED)
#v=np.linalg.cholesky(W_ED)
try:
    v= np.linalg.cholesky(W_ED)
    print("Es positiva")
except np.linalg.LinAlgError:
    v = nearestPD(W_ED)
    print("No Es positiva")
V=v.transpose()
print(V)
print("\n")
r=np.random.random((2*k+1,1))
print("Esto es r:",r)
Atransponer=V[:,0:2*k]
a=Atransponer.transpose()
print("\n")
cosas=np.dot(a,r) 
print("\n")
Btransponer=V[:,2*k]
b=Btransponer.transpose()
cosas1=np.dot(b,r)
print("\n")
Aquantiz=(cosas)/(cosas1)
print("Esto es Aquantiz:",Aquantiz)
print("\n")
randomization=quantiz(Aquantiz,simb)
print(randomization)

No Es positiva
[[ 0.99998125 -1.3035068  -2.99976434  2.88534786  0.9999339 ]
 [-1.3035068   1.69918527  3.91035291 -3.76119403 -1.30346689]
 [-2.99976434  3.91035291  8.9989547  -8.65567565 -2.99967825]
 [ 2.88534786 -3.76119403 -8.65567565  8.32550803  2.88526084]
 [ 0.9999339  -1.30346689 -2.99967825  2.88526084  1.        ]]


Esto es r: [[0.61982542]
 [0.72161322]
 [0.82336413]
 [0.90780977]
 [0.00391546]]






Esto es Aquantiz: [[ 0.99994976]
 [-1.30360556]
 [-3.00012566]
 [ 2.88557162]]


[[ 1.]
 [-1.]
 [-3.]
 [ 3.]]


In [33]:
print("Valor de los símbolos de entrada \n",s)
print("\nValor simple quantization \n", simple_quantiz)
print("\nValor eigenvalue descomposition \n",eigen_descomposition)
print("Valor randomization \n",randomization)

Valor de los símbolos de entrada 
 [[ 1]
 [-1]
 [-3]
 [ 3]]

Valor simple quantization 
 [[ 1.]
 [-1.]
 [-3.]
 [ 3.]]

Valor eigenvalue descomposition 
 [[ 1.]
 [-1.]
 [-3.]
 [ 3.]]
Valor randomization 
 [[ 1.]
 [-1.]
 [-3.]
 [ 3.]]
